# LongRAG example

This LlamaPack implements LongRAG based on [this paper](https://arxiv.org/pdf/2406.15319).

LongRAG retrieves large tokens at a time, with each retrieval unit being ~6k tokens long, consisting of entire documents or groups of documents. This contrasts the short retrieval units (100 word passages) of traditional RAG. LongRAG is advantageous because results can be achieved using only the top 4-8 retrieval units, and long-context LLMs can better understand the context of the documents because long retrieval units preserve their semantic integrity.

## Setup

In [ ]:
%pip install llama-index

## Usage

Below shows the usage of `LongRAGPack` using the `gpt-4o` LLM, which is able to handle long context inputs.

In [ ]:
from llama_index.packs.longrag import LongRAGPack
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI("gpt-4o")

pack = LongRAGPack(data_dir="./data")

In [ ]:
from IPython.display import Markdown, display

query_str = (
    "How can Pittsburgh become a startup hub, and what are the two types of moderates?"
)
res = pack.run(query_str)
display(Markdown(str(res)))

Pittsburgh can become a startup hub by leveraging its increasing population of young people, particularly those aged 25 to 29, who are crucial for the startup ecosystem. The city should encourage the youth-driven food boom, preserve historic buildings, and capitalize on its pre-car city layout to make it more bicycle and pedestrian-friendly. Additionally, Carnegie Mellon University (CMU) should focus on being an even better research university to attract ambitious talent. The city should also foster a culture of tolerance and gradually build an investor community, while startups should aim to become profitable with minimal outside funding initially.

There are two types of moderates: intentional moderates and accidental moderates. Intentional moderates deliberately choose positions midway between the extremes of right and left, while accidental moderates end up in the middle on average because they make up their own minds about each question, finding the far right and far left roughly equally wrong.

Other parameters include `chunk_size`, `similarity_top_k`, and `small_chunk_size`.
- `chunk_size`: To demonstrate how different documents are grouped together, documents are split into nodes of `chunk_size` tokens, then re-grouped based on the relationships between the nodes. Because this does not affect the final answer, it can be disabled by setting `chunk_size` to None. The default size is 4096.
- `similarity_top_k`: Retrieves the top k large retrieval units. The default is 8, and based on the paper, the ideal range is 4-8.
- `small_chunk_size`: To compare similarities, each large retrieval unit is split into smaller child retrieval units of `small_chunk_size` tokens. The embeddings of these smaller retrieval units are compared to the query embeddings. The top k large parent retrieval units are chosen based on the maximum scores of their smaller child retrieval units. The default size is 512.

In [ ]:
pack = LongRAGPack(data_dir="./data", chunk_size=None, similarity_top_k=4)
query_str = (
    "How can Pittsburgh become a startup hub, and what are the two types of moderates?"
)
res = pack.run(query_str)
display(Markdown(str(res)))

Pittsburgh can become a startup hub by leveraging its increasing population of young people, particularly those aged 25 to 29, who are crucial for the startup ecosystem. Encouraging the youth-driven food boom, preserving historic buildings, and making the city more bicycle and pedestrian-friendly are key steps. Additionally, Carnegie Mellon University (CMU) can play a significant role by continuing to be a top-tier research institution and attracting ambitious talent. The city should also foster a culture of tolerance and gradually build an investor community.

The two types of moderates are intentional moderates and accidental moderates. Intentional moderates deliberately choose positions midway between the extremes of right and left, while accidental moderates end up in the middle on average because they make up their own minds about each question, with the far right and far left being roughly equally wrong.

## Vector Storage

The vector index can be extracted and be persisted to disk. A `LongRAGPack` can also be constructed given a vector index. Below is an example of persisting the index to disk.

In [ ]:
import typing as t
from llama_index.core import VectorStoreIndex

modules = pack.get_modules()
index = t.cast(VectorStoreIndex, modules["index"])
index.storage_context.persist(persist_dir="./paul_graham")

Below is an example of loading an index.

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

ctx = StorageContext.from_defaults(persist_dir="./paul_graham")
index = load_index_from_storage(ctx)
pack_from_idx = LongRAGPack(data_dir="./data", index=index)
query_str = (
    "How can Pittsburgh become a startup hub, and what are the two types of moderates?"
)
res = pack.run(query_str)
display(Markdown(str(res)))

To transform Pittsburgh into a startup hub, several strategies can be employed. Encouraging the youth-driven food boom is essential, as it attracts young people, particularly those aged 25 to 29, who are crucial for startups. The city should also focus on maintaining its affordable yet desirable housing, preserving historic buildings, and enhancing its bicycle and pedestrian infrastructure to make it more appealing to young professionals. Additionally, leveraging Carnegie Mellon University's (CMU) research capabilities and fostering a socially liberal culture that tolerates strangeness can further support the startup ecosystem. Although Pittsburgh currently lacks a robust investor community, the decreasing cost of starting startups and alternative funding sources like Kickstarter and Y Combinator can mitigate this challenge over time.

Regarding the two types of moderates, they can be categorized as intentional and accidental moderates. Intentional moderates deliberately choose positions midway between the extremes of right and left, while accidental moderates form their opinions independently on each issue, resulting in a broad range of views that average out to a moderate position. Intentional moderates' opinions are predictable and shift with the median opinion, whereas accidental moderates' opinions are more varied and independent.